We scanned *a lot* of teeth for the ZMK guys and gals.
Let's programmatically generate overview images of them.

In [35]:
import platform
import os
import glob
import pandas
# import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
# import seaborn
import dask
import dask_image.imread
from dask.distributed import Client
client = Client()
# Then go to http://localhost:8787/status
# from numcodecs import Blosc
from tqdm import notebook
# import skimage.filters
# import skimage.morphology
# import skimage.segmentation
# import scipy.ndimage

In [2]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

You can seee what DASK is doing at "http://localhost:8787/status"


In [4]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit

In [5]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [6]:
# Different locations if running either on Linux or Windows
FastSSD = False
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
else:
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('S:\\')
        else:
            BasePath = os.path.join('D:\\Results')
Root = os.path.join(BasePath, 'ZMK')
print('We are loading all the data from %s' % Root)

We are loading all the data from /home/habi/1272/ZMK


In [7]:
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})

In [8]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [9]:
# Look only for folders: https://stackoverflow.com/a/38216530
Data['Folder'] = glob.glob(os.path.join(Root, 'ToothBattallion', '*' + os.path.sep))

In [10]:
print('We found %s tooth folders in %s' % (len(Data), Root))

We found 104 tooth folders in /home/habi/1272/ZMK


In [11]:
# Look for the relevant logfiles
Data['LogFile'] = [sorted(glob.glob(os.path.join(f, '*.log')))[0] for f in Data['Folder']]

In [12]:
Data['Sample'] = [os.path.splitext(os.path.basename(l))[0] for l in Data['LogFile']]

In [13]:
# Proper sorting *with* leading zeros :)
Data.sort_values(by=['Sample'], inplace=True)

In [14]:
# Reconstruction filenames
Data['OutputNameRec'] = [os.path.join(f, sample + '_rec.zarr') for f, sample in zip(Data.Folder, Data.Sample)]

In [15]:
# Load the reconstructions a zarr arrays
Reconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRec']]

In [16]:
# How big are the datasets?
Data['Size'] = [rec.shape for rec in Reconstructions]

In [17]:
# The three cardinal directions
directions = ['Axial',
              'Coronal',
              'Sagittal']

In [18]:
# Cropped filenames
Data['OutputNameRecCrop'] = [os.path.join(f, sample + '_rec_crop.zarr') for f, sample in zip(Data.Folder, Data.Sample)]

In [19]:
# Load the cropped zarr arrays 
CroppedReconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRecCrop']]

In [20]:
# Update the size
Data['Size'] = [rec.shape for rec in CroppedReconstructions]

In [21]:
# Pulpa file names
Data['OutputNamePulpa'] = [os.path.join(f, sample + '_pulpa.zarr') for f, sample in zip(Data.Folder, Data.Sample)]

In [22]:
# Load the cropped zarr arrays back in as *reconstructions*
Pulpas = [dask.array.from_zarr(file) for file in Data['OutputNamePulpa']]

In [23]:
# 3D-Display

In [24]:
import itkwidgets
from itkwidgets import view  # 3d viewer

In [25]:
whichone=74
Data.Sample[whichone]

'Tooth072'

In [29]:
TeethPulpa = numpy.maximum(Pulpas[whichone][1:] * 255, CroppedReconstructions[whichone][1:])

In [31]:
view(TeethPulpa.astype('uint8'))

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC3; proxy …

In [28]:
# view(Reconstructions[whichone])

In [29]:
# view(Pulpas[whichone])

In [32]:
import ipywebrtc as webrtc

In [37]:
import time

In [51]:
viewer = view(test,
              annotations=False,
              ui_collapsed=True,
              rotate=False,
              gradient_opacity=0.618)
viewer

Viewer(annotations=False, geometries=[], gradient_opacity=0.618, point_sets=[], rendered_image=<itkImagePython…

In [52]:
viewer = view(test,
              annotations=False,
              ui_collapsed=True,
              rotate=False,
              gradient_opacity=0.618)
# Rotate camera, based on https://git.io/JfjwT
viewer.camera = numpy.array([viewer.camera[0],
                             viewer.camera[1],
                             [-8.8914531e-01,  4.5637247e-01, -3.3833973e-02]], dtype=numpy.float32)
viewer

Viewer(annotations=False, camera=array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.     …

In [53]:
from IPython.display import display

In [54]:
# for c, rec in notebook.tqdm(enumerate(Reconstructions[:4])):
#     viewer.image = rec[::15,::15,::15].compute()    
#     # Based on https://app.reviewnb.com/InsightSoftwareConsortium/itkwidgets/pull/346/files/
#     # Discussion: https://github.com/InsightSoftwareConsortium/itkwidgets/issues/344
#     recorder
#     recorder.recording = True
#     recorder.recording = False

In [55]:
# Based on https://app.reviewnb.com/InsightSoftwareConsortium/itkwidgets/pull/346/files/
# Discussion: https://github.com/InsightSoftwareConsortium/itkwidgets/issues/344
recorder = webrtc.ImageRecorder(stream=viewer,
                                filename=os.path.join(Data['Folder'][whichone],
                                                      Data['Sample'][whichone] + '_3D.png'),
                                autosave=True)
recorder
recorder.recording = True
recorder.recording = False

In [56]:
# https://github.com/InsightSoftwareConsortium/itkwidgets/issues/344#issuecomment-652735527
from IPython.display import display

In [57]:
display(viewer)
for c, rec in notebook.tqdm(enumerate(Reconstructions[:4])):
    print(Data.Sample[c])
    # downscale
    viewer.image = rec[::10,::10,::10].compute()
    # Based on https://app.reviewnb.com/InsightSoftwareConsortium/itkwidgets/pull/346/files/
    # Discussion: https://github.com/InsightSoftwareConsortium/itkwidgets/issues/344
    recorder = webrtc.ImageRecorder(stream=viewer,
                                    filename=os.path.join(Data['Folder'][c],
                                                          Data['Sample'][c] + '_3D.png'),
                                    autosave=True)
    recorder
    recorder.recording = True
    time.sleep(3)
    recorder.recording = False

Viewer(annotations=False, camera=array([[-1.5463998e+02, -1.5463998e+02, -2.5486960e+02],
       [ 5.4000000e+…

Tooth001
Tooth010
Tooth100
Tooth101



In [ ]:
# # From Kevin: https://twitter.com/maartenbreddels/status/1042005230437756928
# import ipyvolume as ipv
# import numpy as np
# from matplotlib.pyplot import cm

In [ ]:
# # download the dataset
# !wget -q https://www.dropbox.com/s/eqht79b7j4jqit2/petct.npz?dl=1 -O petct.npz

In [ ]:
# full_scan = {k: v.swapaxes(0, 1)[::-1] for k,v in np.load('petct.npz').items()}
# print(list(full_scan.keys()))

In [ ]:
# table_ct = cm.gray_r(np.linspace(0, 1, 255))
# table_ct[:50, 3] = 0 # make the lower values transparent
# table_ct[50:, 3] = np.linspace(0, 0.05, table_ct[50:].shape[0])
# tf_ct = ipv.TransferFunction(rgba=table_ct)

In [ ]:
# ct_vol = ipv.quickvolshow(full_scan['ct_data'], 
#                           tf=tf_ct, lighting=False, 
#                           data_min=-1000, data_max=1000)
# ct_vol

In [ ]:
# import ipywidgets

In [ ]:
# import ipyvolume.pylab as p3
# import ipyvolume as ipv
# import IPython.display
# import ipyvolume
# import numpy as np
# import os

In [ ]:
# ball = ipyvolume.examples.ball(shape=32, draw=False)

In [ ]:
# fig = p3.figure(screen_capture_enabled=True)
# vol = p3.volshow(test)
# p3.show()

In [ ]:
# import k3d

In [ ]:
# tooth = k3d.volume(
#     Reconstructions[1],
# #     alpha_coef=1000,
# #     shadow='dynamic',
# #     samples=600,
# #     shadow_res=128,
# #     shadow_delay=50,
#     color_range=[50,2**8],
# #     color_map=(np.array(k3d.colormaps.matplotlib_color_maps.Gist_heat).reshape(-1,4)
# #                * np.array([1,1.75,1.75,1.75])).astype(np.float32),
# #     compression_level=9
# )
# plot = k3d.plot()
# plot += tooth
# plot.display()

In [ ]:
# with open('tooth001.html','w') as fp:
#     fp.write(plot.snapshot)